In [1]:
import pandas as pd
import pandahouse as ph

In [2]:
connection_to_db = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'project_variant_2',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [3]:
my_query = '''
SELECT
    base.b_purchase_date AS purchase_date,
    base.b_purchase_id AS purchase_id,
    base.b_client_id AS client_id,
    dateDiff('year', client.birth_date, now()) AS client_age,
    dateDiff('year', client.registration, now()) AS client_registration_age,
    CASE
        WHEN base.b_purchase_date = min_date_table.min_date THEN 'new'
        ELSE 'old'
    END AS client_category,
    promotion.promotion_name AS promotion_name,
    promotion.category_name AS category_name,
    promotion.partner_name AS partner_name,
    city.client_city AS client_city,
    city.city AS city,
    base.b_revenue AS revenue,
    base.b_quantity AS quantity
    
FROM    
    (SELECT
        purchase.purchase_date AS b_purchase_date,
        purchase.purchase_id AS b_purchase_id,
        purchase.client_id AS b_client_id,
        purchase.promotion_id AS b_promotion_id,
        purchase.city_id AS b_city_id,
        SUM(CAST(purchase.price AS Float64)) AS b_revenue,
        SUM(CAST(purchase.quantity AS Float64)) AS b_quantity,
        purchase.status AS b_status
        
    FROM project_variant_2.purchase
    
    GROUP BY b_purchase_date, b_purchase_id, b_client_id, b_promotion_id, b_city_id, b_status) AS base

    LEFT JOIN project_variant_2.client AS client
    ON base.b_client_id = client.client_id
    
    LEFT JOIN
        (SELECT
            client_id,
            MIN(purchase_date) AS min_date
        FROM project_variant_2.purchase
        GROUP BY client_id) AS min_date_table
    ON base.b_client_id = min_date_table.client_id
    
    LEFT JOIN project_variant_2.promotion AS promotion
    ON base.b_promotion_id = promotion.promotion_id
    
    LEFT JOIN project_variant_2.city AS city
    ON (base.b_city_id = city.city_id AND client.client_city_id = city.client_city_id)

WHERE
        purchase_date > '2020-05-01'
    AND purchase_date < '2020-08-01'
    AND base.b_status = 1

'''

In [4]:
df = ph.read_clickhouse(query=my_query, connection=connection_to_db)
df.head()

,purchase_date,purchase_id,client_id,client_age,client_registration_age,client_category,promotion_name,category_name,partner_name,client_city,city,revenue,quantity
0,2020-05-04,6787,832,30,2,old,campaign_3,Овощи,Всем партнёрам партнёр,Киев,Казань,110.0,1.0
1,2020-07-04,6503,453,21,1,new,campaign_1,Бакалея,Google,Щучинщина,Вена,235.0,3.0
2,2020-05-22,8137,976,26,2,old,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Вена,Минск,430.0,8.0
3,2020-06-02,1046,593,28,1,old,campaign_3,Овощи,Всем партнёрам партнёр,Казань,Ярославль,220.0,5.0
4,2020-07-31,7139,110,28,2,old,campaign_1,Бакалея,Google,Москва,Санкт-Петербург,115.0,3.0
